In [1]:
import czifile
from skimage import io

In [2]:
path = 'RBC_tiny.czi'

with czifile.CziFile(path) as czi:
    
    
    # Extract raw metadata
    raw_metadata = czi.metadata()

    # If you want the metadata in dictionary form (easier to navigate), you can use:
    metadata_dict = czi.metadata(raw=False)
    channels_info = metadata_dict['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['Channels']
    
print(channels_info['Channel']['Id'])
print(channels_info['Channel']['ExcitationWavelength'])

Channel:0
488


In [3]:
from skimage.io import imread
import pyclesperanto_prototype as cle
import numpy as np

In [4]:
#Read lattice file
rbc = imread("sample_data/RBC_lattice.tif")
voxel_size_x_in_microns = 0.1449922
voxel_size_y_in_microns = 0.1449922
voxel_size_z_in_microns = 0.3

deskewing_angle_in_degrees = 30

print("Shape of original image", rbc.shape )

Shape of original image (150, 118, 209)


In [5]:
#Perform deskewing on rbc image  
deskewed = cle.deskew_y(rbc, 
                        angle_in_degrees=deskewing_angle_in_degrees, 
                        voxel_size_x=voxel_size_x_in_microns, 
                        voxel_size_y=voxel_size_y_in_microns, 
                        voxel_size_z=voxel_size_z_in_microns)

print("Shape of deskewed image",deskewed.shape)

Shape of deskewed image (59, 413, 209)


4 warnings generated.


In [6]:
import napari
results = napari.Viewer()

results.add_image(rbc)

<Image layer 'rbc' at 0x7fc4c29eb730>

In [ ]:
# lls_dd imports
from lls_dd.transform_helpers import *
from lls_dd.transforms import *
## Depending on whether you want to use ndimage or gputools, uncomment one of the following
## two lines
#from scipy.ndimage import affine_transform
from lls_dd.gputools_wrapper import affine_transform_gputools as affine_transform

deskewed = affine_transform(vol, np.linalg.inv(skew), output_shape=output_shape,order=1)



In [6]:
from pycudadecon import affineGPU


/tmp/ipykernel_108150/3539752896.py:1: UserWarning: 
Unable to find library 'lidbcudaDecon'
Please try `conda install -c conda-forge cudadecon`.

Most functionality will fail!

  from pycudadecon import affineGPU


In [7]:
nx, ny, nz = rbc.shape 
voxel_size_xyz=(voxel_size_x_in_microns, 
                voxel_size_y_in_microns, 
                voxel_size_z_in_microns)

T = np.array([[1, 0, 0, nx],

              [0, 1, 0, ny],

              [0, 0, 1, nz],

              [0, 0, 0, 1]])

rotated = affineGPU(rbc, T, voxel_size_xyz)


FileNotFoundError: Unable to find library 'lidbcudaDecon'
Please try `conda install -c conda-forge cudadecon`.